In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
mpl.rc('font', size=12)

from numba import njit

# Voter model with mutation

In [10]:
@njit
def init_grid(L):
    '''Initialises an L by L grid with numbers drawn randomly from (0, 1)'''
    return np.random.rand(L, L)

In [11]:
@njit
def get_4_neighbors(i, j, L):
    '''Finds upper, lower, left and right neighbor of a site in a K by K grid'''
    neighbors = []
    # Up, down, right, left
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  
    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [12]:
@njit
def get_9_neighbors(i, j, L):
    '''Finds all surrounding neighbors of a site in a L by L grid (Moore neighborhood)'''
    neighbors = []

    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, -1), (1, -1), (1, 0), (1, 1)]

    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [38]:
import math

def get_neighbors_fattail(i, j, L, R, eta):
    ''' Computes fat-tailed dispersal kernel
    Parameters:
    - eta: defines "fatness" of the distribution
    - L: length scale or width of kernel
    - R: distance between parent and offspring

    Returns:
    - K: dispersal probability density
    '''

    neighbors = []
    probabilities = []
    directions = []

    # Get coordinates of neighbors in range of R
    for r in range(-R, R + 1):
        for c in range(-R, R + 1):
            
            if r == 0 and c == 0:
                continue

            directions.append((r, c))
    
    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))

        # Calculate Euclidean distance between center and neighbor
        dist = math.dist([i, j], [neighbor_i, neighbor_j])

        # Calculate fat-tail probability
        K = ((eta + 2) / (2 * np.pi * L**2)) * (1 + ((dist)/L)**2 )**(eta/2)
        probabilities.append(K)
        
    probabilities = np.array(probabilities) / sum(probabilities)
    return neighbors, probabilities

In [45]:
get_neighbors_fattail(i, j, L, R, eta)

([(5, 5),
  (5, 6),
  (5, 7),
  (5, 8),
  (5, 9),
  (5, 10),
  (5, 11),
  (5, 12),
  (5, 13),
  (5, 14),
  (5, 15),
  (5, 16),
  (5, 17),
  (5, 18),
  (5, 19),
  (5, 20),
  (5, 21),
  (5, 22),
  (5, 23),
  (5, 24),
  (5, 25),
  (5, 26),
  (5, 27),
  (5, 28),
  (5, 29),
  (5, 30),
  (5, 31),
  (5, 32),
  (5, 33),
  (5, 34),
  (5, 35),
  (5, 36),
  (5, 37),
  (5, 38),
  (5, 39),
  (5, 40),
  (5, 41),
  (5, 42),
  (5, 43),
  (5, 44),
  (5, 45),
  (5, 46),
  (5, 47),
  (5, 48),
  (5, 49),
  (5, 0),
  (5, 1),
  (5, 2),
  (5, 3),
  (5, 4),
  (5, 5),
  (5, 6),
  (5, 7),
  (5, 8),
  (5, 9),
  (5, 10),
  (5, 11),
  (5, 12),
  (5, 13),
  (5, 14),
  (5, 15),
  (5, 16),
  (5, 17),
  (5, 18),
  (5, 19),
  (5, 20),
  (5, 21),
  (5, 22),
  (5, 23),
  (5, 24),
  (5, 25),
  (5, 26),
  (5, 27),
  (5, 28),
  (5, 29),
  (5, 30),
  (5, 31),
  (5, 32),
  (5, 33),
  (5, 34),
  (5, 35),
  (5, 36),
  (5, 37),
  (5, 38),
  (5, 39),
  (5, 40),
  (5, 41),
  (5, 42),
  (5, 43),
  (5, 44),
  (5, 45),
  (5, 46),
  (

In [31]:
import random

# @njit
def voter_model(grid_0, alpha, n_iters, kernel='nearest', eta=None, R=None):
    '''Run experiment with the voter model
    Inputs:
    grid_0 (numpy array): Initial grid
    alpha (float): Value of alpha parameter
    n_iters (int): number of monte carlo steps
    eta (float/int): defines "fatness" of the distribution
    L (int): length scale or width of kernel
    R (int): distance between parent and offspring
    
    Returns:
    cur_grid (numpy array): Grid after n_iter iterations
    num_species (list): Contains amount of different species at each tenth iteration
    '''
    # Create list to store number of unique species
    num_species = []
    cur_grid = np.copy(grid_0)

    height, width = cur_grid.shape

    xs = np.random.choice(np.arange(0, width), size=n_iters)
    ys = np.random.choice(np.arange(0, height), size=n_iters)
        
    # Update grid n_iter times
    for i in range(n_iters):
        if random.random() < alpha:
            cur_grid[xs[i], ys[i]] = random.random()
        # Set species in cell to that in one of its 4 neighbors with equal probability
        else:
            if kernel == 'nearest':
                neighbors = get_4_neighbors(xs[i], ys[i], L)
                neighbor_idx = np.random.choice(len(neighbors))
                neighbor = neighbors[neighbor_idx]
            if kernel == 'fat tail':
                neighbors, probabilities = get_neighbors_fattail(xs[i], ys[i], L, R, eta)
                neighbor_idx = np.random.choice(len(neighbors), p=probabilities)
                neighbor = neighbors[neighbor_idx]
                
            new_type = cur_grid[neighbor]
            cur_grid[xs[i], ys[i]] = new_type
            
        # Save amount of species every tenth iteration
        if i % (n_iters//100) == 0:
            num_species.append(len(np.unique(cur_grid)))
    return cur_grid, num_species

In [32]:
def sa_curve(grid):
    height, width = grid.shape
    
    n_centers = 10
    centers_x = np.random.choice(np.arange(0, width), n_centers) + width
    centers_y = np.random.choice(np.arange(0, height), n_centers) + height

    areas = []
    species = []
    
    torus_grid = np.vstack((grid, grid, grid))
    torus_grid = np.hstack((torus_grid, torus_grid, torus_grid))
    
    torus_grid.shape
    
    for i, (x, y) in enumerate(zip(centers_x, centers_y)):
        cur_species = []
        for j in range(width//2):
            cur_species.append(len(np.unique(torus_grid[x-j:x+j+1, y-j:y+j+1])))
            if i == 0:
                areas.append((j+1)**2)
        species.append(cur_species)
    
    return areas, species